In [ ]:
# MACHINE LEARNING LAB - CSE 3008
#--------------------------------
# NAME: R B Ch S Tarun
# Date: 26 - OCT - 2021
#MOUNTING THE GOOGLE DRIVE TO ACCESSING THE CONTENTS..
from google.colab import drive
drive.mount('/content/gdrive',force_remount = True)

Mounted at /content/gdrive


In [ ]:
# Import numpy and pandas libraries
import numpy;
import pandas as pd;

In [ ]:
# Importing the data from g-drive
data_tennis = pd.read_csv('/content/gdrive/MyDrive/ML_Lab/PlayTennis.csv')
print(data_tennis)

     Outlook Temperature Humidity    Wind Play_Tennis
0      Sunny         Hot     High    Weak          No
1      Sunny         Hot     High  Strong          No
2   Overcast         Hot     High    Weak         Yes
3       Rain        Mild     High    Weak         Yes
4       Rain        Cool   Normal    Weak         Yes
5       Rain        Cool   Normal  Strong          No
6   Overcast        Cool   Normal  Strong         Yes
7      Sunny        Mild     High    Weak          No
8      Sunny        Cool   Normal    Weak         Yes
9       Rain        Mild   Normal    Weak         Yes
10     Sunny        Mild   Normal  Strong         Yes
11  Overcast        Mild     High  Strong         Yes
12  Overcast         Hot   Normal    Weak         Yes
13      Rain        Mild     High  Strong          No


In [ ]:
# Creating Functions to calculate the entropy
def entropy(probs): 
  import math
  return sum( [-prob*math.log(prob, 2) for prob in probs])

def entropy_of_list(a_list): 
  from collections import Counter
  cnt = Counter(x for x in a_list)
  
  num_instances = len(a_list)*1.0
  print("\n Number of Instances of the Current Sub Class is {0}:".format(num_instances ))
  probs = [x / num_instances for x in cnt.values()] 
  print("\n Classes:",min(cnt),max(cnt))
  print(" \n Probabilities of Class {0} is {1}:".format(min(cnt),min(probs)))
  print(" \n Probabilities of Class {0} is {1}:".format(max(cnt),max(probs)))
  return entropy(probs) 
   
print("\n INPUT DATA SET FOR ENTROPY CALCULATION:\n", data_tennis['Play_Tennis'])

entro_total = entropy_of_list(data_tennis['Play_Tennis'])
 
print("\n Total Entropy of PlayTennis Data Set:",entro_total)


 INPUT DATA SET FOR ENTROPY CALCULATION:
 0      No
1      No
2     Yes
3     Yes
4     Yes
5      No
6     Yes
7      No
8     Yes
9     Yes
10    Yes
11    Yes
12    Yes
13     No
Name: Play_Tennis, dtype: object

 Number of Instances of the Current Sub Class is 14.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.35714285714285715:
 
 Probabilities of Class Yes is 0.6428571428571429:

 Total Entropy of PlayTennis Data Set: 0.9402859586706309


In [ ]:

# Function for finding Information Gain
def information_gain(df, split_attribute_name, target_attribute_name, trace=0):
  print("Information Gain Calculation of ",split_attribute_name)
  '''
  Takes a DataFrame of attributes, and quantifies the entropy of a target
  attribute after performing a split along the values of another attribute.
  '''
  df_split = df.groupby(split_attribute_name)
  
  nobs = len(df.index) * 1.0
  
  df_agg_ent = df_split.agg({target_attribute_name : [entropy_of_list, lambda x: len(x)/nobs]})[target_attribute_name]
  
  df_agg_ent.columns = ['Entropy', 'PropObservations']
  
  new_entropy = sum( df_agg_ent['Entropy'] * df_agg_ent['PropObservations'] )
  old_entropy = entropy_of_list(df[target_attribute_name])
  return old_entropy - new_entropy

print('I.G  for Outlook is :'+str( information_gain(data_tennis, 'Outlook', 'Play_Tennis')),"\n")
print('\n I.G  for Humidity is: ' + str( information_gain(data_tennis, 'Humidity', 'Play_Tennis')),"\n")
print('\n I.G  for Wind is:' + str( information_gain(data_tennis, 'Wind', 'Play_Tennis')),"\n")
print('\n I.G  for Temperature is:' + str( information_gain(data_tennis,'Temperature','Play_Tennis')),"\n")

Information Gain Calculation of  Outlook

 Number of Instances of the Current Sub Class is 4.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 14.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.35714285714285715:
 
 Probabilities of Class Yes is 0.6428571428571429:
I.G  for Outlook is :0.2467498197744391 

Information Gain Calculation of  Humidity

 Number of Instances of the Current Sub Class is 7.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.42857142857142855:
 
 Probabilities of Class Yes is 0.5714285714285714:

 Number of Instances of the Current Sub Class is 7.0:

 Classes: No Y

In [ ]:
def id3(df, target_attribute_name, attribute_names, default_class=None):
  
  from collections import Counter
  cnt = Counter(x for x in df[target_attribute_name])
  
  if len(cnt) == 1:
    return next(iter(cnt)) 
  elif df.empty or (not attribute_names):
    return default_class 
  
  else:
      default_class = max(cnt.keys())
      gainz = [information_gain(df, attr, target_attribute_name) for attr in attribute_names] #
      index_of_max = gainz.index(max(gainz))
      best_attr = attribute_names[index_of_max]
      tree = {best_attr:{}} 
      remaining_attribute_names = [i for i in attribute_names if i != best_attr]
      for attr_val, data_subset in df.groupby(best_attr):
        subtree = id3(data_subset,target_attribute_name,remaining_attribute_names,default_class)
      tree[best_attr][attr_val] = subtree
      return tree


In [ ]:
attribute_names = list(data_tennis.columns)
print("List of Attributes:", attribute_names) 
attribute_names.remove('Play_Tennis') 
print("Predicting Attributes:", attribute_names)



List of Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind', 'Play_Tennis']
Predicting Attributes: ['Outlook', 'Temperature', 'Humidity', 'Wind']


In [ ]:
# Displaying the Information
from pprint import pprint
tree = id3(data_tennis,'Play_Tennis',attribute_names)
print("\n\nThe Resultant Decision Tree is :\n")
pprint(tree)
attribute = next(iter(tree))
print("Best Attribute :\n",attribute)
print("Tree Keys:\n",tree[attribute].keys())

Information Gain Calculation of  Outlook

 Number of Instances of the Current Sub Class is 4.0:

 Classes: Yes Yes
 
 Probabilities of Class Yes is 1.0:
 
 Probabilities of Class Yes is 1.0:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 5.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.4:
 
 Probabilities of Class Yes is 0.6:

 Number of Instances of the Current Sub Class is 14.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.35714285714285715:
 
 Probabilities of Class Yes is 0.6428571428571429:
Information Gain Calculation of  Temperature

 Number of Instances of the Current Sub Class is 4.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.25:
 
 Probabilities of Class Yes is 0.75:

 Number of Instances of the Current Sub Class is 4.0:

 Classes: No Yes
 
 Probabilities of Class No is 0.5:
 
 Probabilities of Class Ye

In [ ]:
def classify(instance, tree, default=None):
  attribute = next(iter(tree)) 
  print("Key:",tree.keys()) 
  print("Attribute:",attribute)
  if instance[attribute] in tree[attribute].keys():
    result = tree[attribute][instance[attribute]]
    print("Instance Attribute:",instance[attribute],"TreeKeys :",tree[attribute].keys())
    if isinstance(result, dict): 
      return classify(instance, result)
    else:
      return result 
  else:
    return default

In [ ]:
data_tennis['predicted'] = data_tennis.apply(classify, axis=1, args=(tree,'No') ) 
print(data_tennis['predicted'])
print('\n Accuracy is:\n' + str( sum(data_tennis['Play_Tennis']==data_tennis['predicted'] ) / 
(1.0*len(data_tennis.index)) ))
data_tennis[['Play_Tennis', 'predicted']]

Key: dict_keys(['Outlook'])
Attribute: Outlook
Instance Attribute: Sunny TreeKeys : dict_keys(['Sunny'])
Key: dict_keys(['Humidity'])
Attribute: Humidity
Key: dict_keys(['Outlook'])
Attribute: Outlook
Instance Attribute: Sunny TreeKeys : dict_keys(['Sunny'])
Key: dict_keys(['Humidity'])
Attribute: Humidity
Key: dict_keys(['Outlook'])
Attribute: Outlook
Key: dict_keys(['Outlook'])
Attribute: Outlook
Key: dict_keys(['Outlook'])
Attribute: Outlook
Key: dict_keys(['Outlook'])
Attribute: Outlook
Key: dict_keys(['Outlook'])
Attribute: Outlook
Key: dict_keys(['Outlook'])
Attribute: Outlook
Instance Attribute: Sunny TreeKeys : dict_keys(['Sunny'])
Key: dict_keys(['Humidity'])
Attribute: Humidity
Key: dict_keys(['Outlook'])
Attribute: Outlook
Instance Attribute: Sunny TreeKeys : dict_keys(['Sunny'])
Key: dict_keys(['Humidity'])
Attribute: Humidity
Instance Attribute: Normal TreeKeys : dict_keys(['Normal'])
Key: dict_keys(['Outlook'])
Attribute: Outlook
Key: dict_keys(['Outlook'])
Attribute: Out

,Play_Tennis,predicted
0,No,None
1,No,None
2,Yes,No
3,Yes,No
4,Yes,No
5,No,No
6,Yes,No
7,No,None
8,Yes,Yes
9,Yes,No
